In [14]:
import requests

def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [15]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

# Now we are importing all the video game companies and looking at the ones in San Francisco 

In [16]:
import re
n = re.compile("^San Francisco",re.IGNORECASE)
gaming_companies = db.video_game_companies.find({"offices_city":n})

In [17]:
import pandas as pd
from pandas.io.json import json_normalize

df = pd.DataFrame(json_normalize(gaming_companies))
df

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,_id,name,category_code,offices_description,offices_address1,offices_address2,offices_zip_code,offices_city,offices_state_code,offices_country_code,offices_latitude,offices_longitude
0,5ef92ffbf4910f763f438bad,Ustream,games_video,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480
1,5ef92ffbf4910f763f438bb1,CastTV,games_video,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913
2,5ef92ffbf4910f763f438bb5,Revision3,games_video,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243
3,5ef92ffbf4910f763f438bb8,blinkx,games_video,None,1 Market Street,"19th Floor, Spear Tower",94105,San Francisco,CA,USA,37.793819,-122.395089
4,5ef92ffbf4910f763f438bbc,hi5,games_video,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558
...,...,...,...,...,...,...,...,...,...,...,...,...
62,5ef92ffbf4910f763f438f36,Bigpoint,games_video,Bigpoint Inc.,500 Howard St.,Suite 300,CA 94105,San Francisco,CA,USA,NaN,NaN
63,5ef92ffbf4910f763f438f3c,Exent,games_video,Premium Services,685 Market Street,Suite 560,94105,San Francisco,CA,USA,37.787646,-122.402759
64,5ef92ffbf4910f763f438f3e,Mondo Media,games_video,Headquarters,550 15TH ST,STE 31,94103,San Francisco,CA,USA,37.764169,-122.402021
65,5ef92ffbf4910f763f438f53,DJ Nitrogen,games_video,,11 Lupine Avenue,,94118,San Francisco,CA,USA,37.784174,-122.449408


In [18]:
import numpy as np

def transformToGeoPoint(s):
    if np.isnan(s.offices_latitude) or np.isnan(s.offices_longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.offices_longitude, s.offices_latitude]
    }
    

df["geopoint"] = df.apply(transformToGeoPoint, axis=1)

## This is the San Francisco Video Game companies with the geopoint column added

In [19]:
df.head(6)

,_id,name,category_code,offices_description,offices_address1,offices_address2,offices_zip_code,offices_city,offices_state_code,offices_country_code,offices_latitude,offices_longitude,geopoint
0,5ef92ffbf4910f763f438bad,Ustream,games_video,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ..."
1,5ef92ffbf4910f763f438bb1,CastTV,games_video,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
2,5ef92ffbf4910f763f438bb5,Revision3,games_video,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,"{'type': 'Point', 'coordinates': [-122.388243,..."
3,5ef92ffbf4910f763f438bb8,blinkx,games_video,None,1 Market Street,"19th Floor, Spear Tower",94105,San Francisco,CA,USA,37.793819,-122.395089,"{'type': 'Point', 'coordinates': [-122.395089,..."
4,5ef92ffbf4910f763f438bbc,hi5,games_video,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."
5,5ef92ffbf4910f763f438bbf,Curse,games_video,San Francisco,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,"{'type': 'Point', 'coordinates': [-122.399972,..."


## Now let's export the previous DataFrame to a database in compass so that we can use the $near operator

In [33]:
df["geopoint"] = df.apply(transformToGeoPoint, axis=1)
df["_id"] = df["_id"].apply(lambda e: str(e))
df

,_id,name,category_code,offices_description,offices_address1,offices_address2,offices_zip_code,offices_city,offices_state_code,offices_country_code,offices_latitude,offices_longitude,geopoint
0,5ef92ffbf4910f763f438bad,Ustream,games_video,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ..."
1,5ef92ffbf4910f763f438bb1,CastTV,games_video,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
2,5ef92ffbf4910f763f438bb5,Revision3,games_video,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,"{'type': 'Point', 'coordinates': [-122.388243,..."
3,5ef92ffbf4910f763f438bb8,blinkx,games_video,None,1 Market Street,"19th Floor, Spear Tower",94105,San Francisco,CA,USA,37.793819,-122.395089,"{'type': 'Point', 'coordinates': [-122.395089,..."
4,5ef92ffbf4910f763f438bbc,hi5,games_video,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,5ef92ffbf4910f763f438f36,Bigpoint,games_video,Bigpoint Inc.,500 Howard St.,Suite 300,CA 94105,San Francisco,CA,USA,NaN,NaN,None
63,5ef92ffbf4910f763f438f3c,Exent,games_video,Premium Services,685 Market Street,Suite 560,94105,San Francisco,CA,USA,37.787646,-122.402759,"{'type': 'Point', 'coordinates': [-122.402759,..."
64,5ef92ffbf4910f763f438f3e,Mondo Media,games_video,Headquarters,550 15TH ST,STE 31,94103,San Francisco,CA,USA,37.764169,-122.402021,"{'type': 'Point', 'coordinates': [-122.402021,..."
65,5ef92ffbf4910f763f438f53,DJ Nitrogen,games_video,,11 Lupine Avenue,,94118,San Francisco,CA,USA,37.784174,-122.449408,"{'type': 'Point', 'coordinates': [-122.4494082..."


In [34]:
#df.to_json("geopoint_added_video_game_companies.json",orient="records")

#### Using the "near" operator to see which video_game company is closest to the warriors stadium, which is known as the Chase Center

In [35]:
financial_district = geocode("Financial district, San Francisco")
financial_district



{'type': 'Point', 'coordinates': [-84.0728, 9.9417]}

In [36]:
def geoQueryNear(point,radius=10000):
    return {
        "geopoint":{
            "$near": {
                "$geometry": point,
                "$maxDistance": radius,
                "$minDistance": 0
            }
        }
    }

In [37]:
q = db.geopoint_added_video_game_companies.find(geoQueryNear(financial_district, radius=10000),{"_id":0,"company_id":0})
near_vg_companies = list(q)
near_vg_companies

[]

## Now let's do the same for design companies in San Francisco which as we saw in our city-lookup jupyter there is only 1. It is the HQ for 99 Designs

I am going to use a specific query to the companies database looking for the latitude and longitude of their offices since i am unable to do so from the design_companies database I created.
Let's get the coordinates of their HQ office through a regular query

In [ ]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [ ]:
query = {
    "$and":[
        {"name":{"$eq":"99designs"}},
        {"offices.city":{"$eq":"San Francisco"}},
        {"offices.description":{"$eq":"United States (HQ)"}}
    ]
}

cur=list(db.companies.find(query,{"name":1, "offices.latitude":1, "offices.longitude":1}))
cur

## We now know that the coordinates for the design office in Califorinia are the following:
'coordinates': [37.7955307,-122.4005983]

In [ ]:
times_square = geocode("Times Square, New York")
times_square